In [1]:
from evaluate import main, run_single
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path="config")


/home/carla/.conda/envs/mile/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


hydra.initialize()

In [ ]:
from evaluate import run_single
import gym
import json
from pathlib import Path
import wandb
import hydra
from omegaconf import DictConfig, OmegaConf
import logging
import os.path
import sys
import cv2
from time import time

from gym.wrappers.monitoring.video_recorder import ImageEncoder
from stable_baselines3.common.vec_env.base_vec_env import tile_images

from carla_gym.utils import config_utils
from utils import server_utils
from mile.agents.rl_birdview.utils.wandb_callback import WandbCallback
from mile.constants import CARLA_FPS
import time


def main(cfg: DictConfig):
    server_utils.kill_carla(cfg.port)

    # start carla servers
    server_manager = server_utils.CarlaServerManager(cfg.carla_sh_path, port=cfg.port)
    server_manager.start()

    # single actor, place holder for multi actors
    agents_dict = {}
    obs_configs = {}
    reward_configs = {}
    terminal_configs = {}
    agent_names = []
    for ev_id, ev_cfg in cfg.actors.items():
        agent_names.append(ev_cfg.agent)
        cfg_agent = cfg.agent[ev_cfg.agent]
        OmegaConf.save(config=cfg_agent, f='config_agent.yaml')
        AgentClass = config_utils.load_entry_point(cfg_agent.entry_point)
        agents_dict[ev_id] = AgentClass('config_agent.yaml')
        obs_configs[ev_id] = agents_dict[ev_id].obs_configs

        # get obs_configs from agent
        reward_configs[ev_id] = OmegaConf.to_container(ev_cfg.reward)
        terminal_configs[ev_id] = OmegaConf.to_container(ev_cfg.terminal)

    env_idx = 0

    ckpt_task_idx = 0
    log.info(f'Start new env from task_idx {ckpt_task_idx}')

    # compose suite_name
    env_setup = OmegaConf.to_container(cfg.test_suites[env_idx])
    suite_name = '-'.join(agent_names) + '_' + env_setup['env_id']
    for k in sorted(env_setup['env_configs']):
        suite_name = suite_name + '_' + str(env_setup['env_configs'][k])

    print(f"Start Benchmarking! env_idx: {env_idx}, suite_name: {suite_name}")

    agents_log_dir = Path('agents_log') / suite_name
    agents_log_dir.mkdir(parents=True, exist_ok=True)

    env = gym.make(env_setup['env_id'], obs_configs=obs_configs, reward_configs=reward_configs,
                   terminal_configs=terminal_configs, host=cfg.host, port=cfg.port,
                   seed=cfg.seed, no_rendering=cfg.no_rendering, **env_setup['env_configs'])

    task_idx = 0
    env.set_task_idx(task_idx)
    run_name = f"{env.task['weather']}_{env.task['route_id']:02d}"
    list_render, ep_stat_dict, ep_event_dict, timestamp = run_single(
        run_name, env, agents_dict, agents_log_dir, cfg.log_video)

    # close env
    env.close()
    env = None
    server_manager.stop()

        

cfg = compose(config_name="evaluate", overrides=[
    "carla_sh_path=/home/carla/CarlaUE4.sh", "agent.mile.ckpt=/home/carla/mile.ckpt", "port=2000"])
main(cfg)


2000/tcp:           


  14254.26.2-0+++UE4+Release-4.26 522 0
Disabling core dumps.
Warning - the cfg_dict merging into the main cfg has keys that do not exist in main: ['DATASET.FREQUENCY', 'EVAL.CHECKPOINT_PATH', 'LOSSES.WEIGHT_REWARD', 'MODEL.POLICY', 'MODEL.REWARD']
Loaded weights from: /home/carla/mile.ckpt
Loading world model weights from /home/carla/mile.ckpt
Recomputing
Start Benchmarking! env_idx: 0, suite_name: mile_LeaderBoard-v0_Town02_None_new


/home/carla/.conda/envs/mile/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
